In [ ]:
!pip install -q sentence-transformers datasets accelerate huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!wget -q https://raw.githubusercontent.com/pierrealexandreguillemin-a11y/pocket_arbiter/main/data/training/triplets_training.jsonl
import json
triplets = [json.loads(l) for l in open("triplets_training.jsonl") if l.strip()]
print(len(triplets))

In [ ]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.losses import MultipleNegativesRankingLoss
from datasets import Dataset

model = SentenceTransformer("google/embeddinggemma-300m-qat-q4_0-unquantized")
trainer = SentenceTransformerTrainer(
    model=model,
    args=SentenceTransformerTrainingArguments(output_dir="embeddinggemma-chess-fr", num_train_epochs=3, per_device_train_batch_size=16, fp16=True, report_to="none"),
    train_dataset=Dataset.from_list(triplets),
    loss=MultipleNegativesRankingLoss(model)
)
trainer.train()
model.save("embeddinggemma-chess-fr")

In [ ]:
import shutil
from google.colab import files
shutil.make_archive("embeddinggemma-chess-fr", "zip", "embeddinggemma-chess-fr")
files.download("embeddinggemma-chess-fr.zip")